In [13]:
#Importing libraries
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np

In [14]:
#Model for image classification
model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),        # prevent overfitting by reducing the model's reliance on any one feature
    Dense(50, activation='relu'),
    Dense(2, activation='softmax') # Used in multiclass classification problems to convert the output of the model into a probability distribution over the classes.
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dropout (Dropout)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 50)                6

In [15]:
#Compiling the model Optimizer, Loss and Accuracy Metrics
model.compile(optimizer='adam', 
              loss='binary_crossentropy', # minimize this difference
              metrics=['acc']) # measures the percentage of correct predictions

In [16]:
#Setting up an image data generator for training the model
TRAINING_DIR = "D:/CVPR/CVPR2/Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, #number of samples in each batch
                                                    target_size=(150, 150)) #resizes the images to 150x150 pixels

Found 3268 images belonging to 2 classes.


In [17]:
#Setting up an image data generator for validation data to evaluate the performance of the model.
VALIDATION_DIR = "D:/CVPR/CVPR2/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255) #image transformation,Normalizing by rescaling the pixel values to be between 0 and 1

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 818 images belonging to 2 classes.


In [18]:
x_train, y_train = train_generator.next()

In [19]:
file = open(filepath+"train_datagen", 'rb')
train_datagen = pickle.load(file)
file.close()
import pickle

filepath = "Pickle Data/"
file = open(filepath+"validation_datagen", 'rb')
train_datagen = pickle.load(file)
file.close()

In [20]:
#Model Fitting
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint]) # After each epoch,the ModelCheckpoint callback will save the model if its validation loss is lower than the previous best. 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 27/327 [=>............................] - ETA: 3:34 - loss: 0.9228 - acc: 0.6630

c:\Users\Asus\anaconda3\envs\cvpr\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


327/327 [==============================] - ETA: 0s - loss: 0.5115 - acc: 0.7840WARNING:tensorflow:From c:\Users\Asus\anaconda3\envs\cvpr\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model2-001.model\assets
327/327 [==============================] - 257s 787ms/step - loss: 0.5115 - acc: 0.7840 - val_loss: 0.4383 - val_acc: 0.8631
Epoch 2/10
327/327 [==============================] - 270s 826ms/step - loss: 0.4167 - acc: 0.8375 - val_loss: 0.2788 - val_acc: 0.9120
Epoch 3/10
327/327 [==============================] - 314s 960ms/step - loss: 0.3739 - acc: 0.8433 - val_loss: 0.237

In [21]:
#Saving the model
model.save('Face_mask_detection_model_2.2.h5')

In [1]:
#Taking live input through camera to detect face mask
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Path to the model file
model_path = "./Face_mask_detection_model_2.2.h5"

# Load the model
model = load_model(model_path)

labels_dict = {0: 'No Mask', 1: 'Mask on'}
color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}

size = 4
webcam = cv2.VideoCapture(0)  # Use camera 0

# Load the pre-trained classifier used by OpenCV to detect faces.
classifier = cv2.CascadeClassifier("D:/CVPR/CVPR2/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml")

while True:
    (rval, im) = webcam.read()
    im = cv2.flip(im, 1, 1)  # Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # Detect faces
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f]  # Scale the shape
        # Adjust the size of the detection box
        detection_box_size = int(max(w, h) * 1.2)
        x -= (detection_box_size - w) // 2
        y -= (detection_box_size - h) // 2
        w = h = detection_box_size
        
        # Make sure the box doesn't go outside the image boundaries
        x = max(0, x)
        y = max(0, y)
        w = min(im.shape[1] - x, w)
        h = min(im.shape[0] - y, h)
        
        face_img = im[y:y + h, x:x + w]
        resized = cv2.resize(face_img, (150, 150))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 150, 150, 3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(im, (x, y), (x + w, y + h), color_dict[label], 2)
        cv2.rectangle(im, (x, y - 40), (x + w, y), color_dict[label], -1)
        cv2.putText(im, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Show the image
    cv2.imshow('Mask_Detection', im)
    key = cv2.waitKey(10)
    # if q key is pressed, break out of the loop
    if key == 113:  # The q key
        break

# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

